# PLSA 5 topics news 20

In [1]:
import pandas as pd
gntm=pd.read_csv("../nb_outputs/news_20/gntm_topic_5.csv")
gntm.drop(["Unnamed: 0"],axis=1,inplace=True)

plsa_5top=pd.read_csv("../plsa_outputs/news_20/5_topics.csv")
plsa_5top.drop(["Unnamed: 0"],axis=1,inplace=True)


c:\anaconda\envs\machine_learning\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\anaconda\envs\machine_learning\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
c:\anaconda\envs\machine_learning\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [3]:
plsa_5top.shape,gntm.shape
gntm

,0,1,2,3,4
0,0.105263,0.210526,0.210526,0.263158,0.210526
1,0.000000,0.250000,0.250000,0.250000,0.250000
2,0.000000,0.333333,0.000000,0.333333,0.333333
3,0.000000,0.000000,1.000000,0.000000,0.000000
4,0.333333,0.266667,0.066667,0.066667,0.266667
...,...,...,...,...,...
10738,0.000000,0.333333,0.333333,0.000000,0.333333
10739,0.200000,0.200000,0.200000,0.200000,0.200000
10740,0.000000,0.250000,0.250000,0.250000,0.250000
10741,0.260870,0.260870,0.130435,0.130435,0.217391


In [4]:
import pickle
with open("../nb_outputs/news_20/missing_values_5.pkl","rb") as f:
    missed_idx=pickle.load(f) #got this from nb_gnn 
# missed_idx #drop these values from plsa  #these are the index which are not in the baysian_topic_5 dataset 

In [5]:
# len(news_20)-len(missed_idx),len(bay)

In [6]:
news_20=pd.read_csv("../datasets/20news.csv")
# print(news_20.shape,bay.shape[0]+len(missed_idx))
#now using this information generate new indexes
#from the original subtract(eliminate) the missing idx and the new idx 
original_idx=list(range(news_20.shape[0]))
# print(len(original_idx))
for item in missed_idx:
    original_idx.remove(item)
# len(original_idx),bay.shape

In [7]:
# make a new column in bayes and append these indexes
bay["index"]=original_idx

# Now remove the rows which was not common to both the datasets

In [8]:
bay_idx,plsa_5top_idx=set(bay["index"]),set(plsa_5top["index"])

In [9]:
idx_to_keep=bay_idx.intersection(plsa_5top_idx)

In [10]:
len(idx_to_keep)

10736

In [11]:
cleaned_bay=bay[bay["index"].isin(idx_to_keep)]
cleaned_plsa_5=plsa_5top[plsa_5top["index"].isin(idx_to_keep)]


In [12]:
temp=cleaned_plsa_5["index"].to_list()
for i in cleaned_bay["index"].to_list():
    if i not in temp:
        print(i,"what the fuck")
    # break

In [13]:
cleaned_bay.shape,cleaned_plsa_5.shape

((10736, 6), (10736, 6))

In [14]:
cleaned_bay.index=cleaned_bay["index"]
cleaned_plsa_5.index=cleaned_plsa_5["index"]

In [15]:
cleaned_plsa_5.drop(["index"],axis=1,inplace=True)
cleaned_bay.drop(["index"],axis=1,inplace=True)

C:\Users\rbw19\AppData\Local\Temp\ipykernel_18060\2972468246.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_plsa_5.drop(["index"],axis=1,inplace=True)
C:\Users\rbw19\AppData\Local\Temp\ipykernel_18060\2972468246.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_bay.drop(["index"],axis=1,inplace=True)


In [16]:
# cleaned_bay

In [17]:
# cleaned_plsa_5

In [18]:
cleaned_plsa_5.columns=[str(i) for i in range(cleaned_plsa_5.shape[1])]

# Finding match accuracy without order

In [23]:
import pandas as pd

def accuracy(n_largest,match_threshold,no_topics): #instead of gntm here we have PLSA or other probabilitic models

    bay=cleaned_bay
    gnt=cleaned_plsa_5 #changing the gntm to plsa
    print("read sucessfully")
    # bay.drop(["Unnamed: 0"],axis=1,inplace=True) 
    # gnt.drop(["Unnamed: 0"],axis=1,inplace=True) 
    bay,gnt=bay.transpose(),gnt.transpose()
    bay_top_3,gnt_top_3=[],[]

    for col in range(bay.shape[1]):
        try:
            bay_top_3.append(bay.nlargest(n_largest,col).index) 
        except:
            pass

    for col in range(gnt.shape[1]):
        try:
            gnt_top_3.append(gnt.nlargest(n_largest,col).index)
        except:
            pass

    bay_col_names=[]
    gnt_col_names=[]
    for i in range(1,n_largest+1):
        bay_col_names.append(f'bay_top{str(i)}')
        gnt_col_names.append(f'gnt_top{str(i)}')
        
    bay_top3_df=pd.DataFrame(bay_top_3,columns=bay_col_names)
    gnt_top3_df=pd.DataFrame(gnt_top_3,columns=gnt_col_names)
    final_matches=[]

    for gnt_out,bay_out in zip(gnt_top_3,bay_top_3):
        matches=[]
        for out in gnt_out:
            if out in bay_out:
                matches.append(True)
            else:
                matches.append(False)
        percentage_of_acc=(matches.count(True)/len(matches))*100

        if percentage_of_acc >= match_threshold: 
            final_matches.append(True)
        else:
            final_matches.append(False)
            
    result=pd.concat([bay_top3_df,gnt_top3_df,pd.DataFrame(final_matches,columns=["match"])],axis=1)
    # decision=str(input("Do you want to save the o/p(yes/no)?"))
    decision="yes"
    if decision=="Yes" or decision=="yes" or decision=="YES":
        file_name=str(input("enter file name :"))
        result.to_csv(f"{file_name}.csv")
    
    try:
        result["match"].value_counts()[True]
    except:
        print('accuracy : 0 % ')
        
    try:
        result["match"].value_counts()[False]
    except:
        print('accuracy : 100%')

    try:
        accuracy=result["match"].value_counts()[True]/(result["match"].value_counts()[True]+result["match"].value_counts()[False])
        print("The accuracy is :",accuracy)
        with open(f"{no_topics}_plsa.txt","a") as op:
            op.writelines(f"\n Number of topics : {no_topics} \n N-largest : {n_largest} \n Match Threshold :{match_threshold} \n accuracy : {accuracy}")
    except:
        print("result already shown")



topics=str(input("enter no. of topics :"))
n_largest=int(input("enter n_largest : "))

thres=[60,80,100]

for threshold in thres:
    print("topics:",topics)
    print("n largest :",n_largest)
    print("Threshold is :",threshold)
    accuracy(n_largest,threshold,topics)

topics: 5
n largest : 5
Threshold is : 60
read sucessfully
accuracy : 100%
result already shown
topics: 5
n largest : 5
Threshold is : 80
read sucessfully
accuracy : 100%
result already shown
topics: 5
n largest : 5
Threshold is : 100
read sucessfully
accuracy : 100%
result already shown


# Problem solvong area

In [ ]:
import pandas as pd


n_largest=2
threshold=60
bay=cleaned_bay
gnt=cleaned_plsa_5 #changing the gntm to plsa
print("read sucessfully")
# bay.drop(["Unnamed: 0"],axis=1,inplace=True) 
# gnt.drop(["Unnamed: 0"],axis=1,inplace=True) 
bay,gnt=bay.transpose(),gnt.transpose()
bay_top_3,gnt_top_3=[],[]
for col in range(bay.shape[1]):
    try:
        bay_top_3.append(bay.nlargest(n_largest,col).index) 
    except:
        pass
for col in range(gnt.shape[1]):
    try:
        gnt_top_3.append(gnt.nlargest(n_largest,col).index)
    except:
        pass
bay_col_names=[]
gnt_col_names=[]
for i in range(1,n_largest+1):
    bay_col_names.append(f'bay_top{str(i)}')
    gnt_col_names.append(f'gnt_top{str(i)}')
    
gnt_top_3



In [ ]:
bay_top3_df=pd.DataFrame(bay_top_3,columns=bay_col_names)
gnt_top3_df=pd.DataFrame(gnt_top_3,columns=gnt_col_names)

gnt_top3_df

In [ ]:
match_threshold=60

In [ ]:
final_matches=[]
for gnt_out,bay_out in zip(gnt_top_3,bay_top_3):
    matches=[]
    for out in gnt_out:
        if out in bay_out:
            matches.append(True)
        else:
            matches.append(False)
    percentage_of_acc=(matches.count(True)/len(matches))*100
    if percentage_of_acc >= match_threshold: 
        final_matches.append(True)
    else:
        final_matches.append(False)
        
result=pd.concat([bay_top3_df,gnt_top3_df,pd.DataFrame(final_matches,columns=["match"])],axis=1)

result

decision=str(input("Do you want to save the o/p(yes/no)?"))
decision="yes"
if decision=="Yes" or decision=="yes" or decision=="YES":
    file_name=str(input("enter file name :"))
    result.to_csv(f"{file_name}.csv")

try:
    result["match"].value_counts()[True]
except:
    print('accuracy : 0 % ')
try:
    result["match"].value_counts()[False]
except:
    print('accuracy : 100%')
try:
    accuracy=result["match"].value_counts()[True]/(result["match"].value_counts()[True]+result["match"].value_counts()[False])
    print("The accuracy is :",accuracy)
except:
    print("result already shown")
# topics=str(input("enter no. of topics :"))
# n_largest=int(input("enter n_largest : "))


# problem in py file

In [102]:
cleaned_plsa_5.head(5)

,topic_0,topic_1,topic_2,topic_3,topic_4
index,,,,,
2.0,0.459501,0.119614,0.231753,0.144837,0.044295
3.0,0.350112,0.156945,0.177958,0.263452,0.051532
4.0,0.116624,0.208942,0.111421,0.520003,0.043009
5.0,0.130336,0.411588,0.196238,0.143387,0.118451
7.0,0.180428,0.059939,0.065561,0.634044,0.060029


In [103]:
cleaned_bay.head(5)

,0,1,2,3,4
index,,,,,
2,0.090909,0.181818,0.181818,0.363636,0.181818
3,0.111111,0.222222,0.222222,0.222222,0.222222
4,0.125000,0.250000,0.125000,0.250000,0.250000
5,0.166667,0.166667,0.333333,0.166667,0.166667
7,0.470588,0.235294,0.029412,0.029412,0.235294


In [ ]:
cleaned_plsa_5.head(5)

In [100]:


bay=cleaned_bay
plsa=cleaned_plsa_5 #changing the gntm to plsa
print("read sucessfully")
# bay.drop(["Unnamed: 0"],axis=1,inplace=True) 
# gnt.drop(["Unnamed: 0"],axis=1,inplace=True) 
bay,plsa=bay.transpose(),plsa.transpose()
bay_top_3,plsa_top_3=[],[]
for col in range(bay.shape[1]):
    try:
        bay_top_3.append(bay.nlargest(n_largest,col).index) 
    except:
        pass
for col in range(plsa.shape[1]):
    try:
        plsa_top_3.append(plsa.nlargest(n_largest,col).index)
    except:
        pass
bay_col_names=[]
plsa_col_names=[]
for i in range(1,n_largest+1):
    bay_col_names.append(f'bay_top{str(i)}')
    plsa_col_names.append(f'plsa_top{str(i)}')
    
bay_top3_df=pd.DataFrame(bay_top_3,columns=bay_col_names)
plsa_top3_df=pd.DataFrame(plsa_top_3,columns=plsa_col_names)


read sucessfully


In [105]:
plsa_top3_df

,bay_top1,bay_top2
0,3,1
1,1,2
2,1,3
3,2,0
4,0,1
...,...,...
6080,0,1
6081,3,0
6082,2,0
6083,2,4


In [106]:
final_matches=[]
for plsa_out,bay_out in zip(plsa_top_3,bay_top_3):
    matches=[]
    for out in plsa_out:
        if out in bay_out:
            matches.append(True)
        else:
            matches.append(False)
    percentage_of_acc=(matches.count(True)/len(matches))*100
    if percentage_of_acc >= match_threshold: 
        final_matches.append(True)
    else:
        final_matches.append(False)
        
result=pd.concat([bay_top3_df,plsa_top3_df,pd.DataFrame(final_matches,columns=["match"])],axis=1)


In [107]:
# decision=str(input("Do you want to save the o/p(yes/no)?"))
decision="yes"
if decision=="Yes" or decision=="yes" or decision=="YES":
    file_name=str(input("enter file name :"))
    result.to_csv(f"{file_name}.csv")

try:
    result["match"].value_counts()[True]
except:
    print('accuracy : 0 % ')
try:
    result["match"].value_counts()[False]
except:
    print('accuracy : 100%')
try:
    accuracy=result["match"].value_counts()[True]/(result["match"].value_counts()[True]+result["match"].value_counts()[False])
    print("The accuracy is :",accuracy)
    with open(f"{no_topics}_plsa.txt","a") as op:
        op.writelines(f"\n Number of topics : {no_topics} \n N-largest : {n_largest} \n Match Threshold :{match_threshold} \n accuracy : {accuracy}")
except:
    print("result already shown")

accuracy : 0 % 
result already shown
